In [1]:
!pip install -qU langchain==0.1.0 langchain-google-genai==0.0.9 langchain-community==0.0.19 python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.3.8 requires langchain-core<1.0.0,>=0.3.51, but you have langchain-core 0.1.23 which is incompatible.
langchain-openai 0.2.8 requires langchain-core<0.4.0,>=0.3.17, but you have langchain-core 0.1.23 which is incompatible.
langchain-experimental 0.3.3 requires langchain-community<0.4.0,>=0.3.0, but you have langchain-community 0.0.19 which is incompatible.
langchain-experimental 0.3.3 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.23 which is incompatible.


In [4]:
import os
#from google.colab import userdata
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# 🔑 SETUP API KEY
os.environ["GOOGLE_API_KEY"] = ""
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1,convert_system_message_to_human=True)

print("✅ Libraries installed and LLM ready.")

✅ Libraries installed and LLM ready.


In [5]:
# Centralized Archiecture : A single agent has access to multiple tools

# Step 1 : Define the tools [workers]
def product_search(query):
  "Worker 1 : the product expert"
  return "Found: iphone 15 (80,000 INR), Samsung S24 (75000 INR), all in stock."

def policy_search(query):
  "Worker 2 : The policy expert"
  return "Policy : 7-day return for phones, no returns for accessories."

# Step 2 : Wrap them as langchain tools!
product_tool = Tool(
    name = "product_search",
    func=product_search,
    description="useful for when you want to know price, stock"
)

policy_tool = Tool(
    name = "policy_search",
    func=policy_search,
    description="useful for when you want to know return policy"
)

In [6]:
# Step 3 : Define the manager / orchestrator
def setup_centralized_agent():
  tools = [product_tool,policy_tool]

  agent = initialize_agent(tools=tools,
                           llm=llm,
                           verbose=True,
                           agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)
  return agent

In [7]:
# Step 4 :
manager = setup_centralized_agent()

/Users/devenderswami/lc-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
manager.run("Whats the price of iphone 15?")

/Users/devenderswami/lc-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "product_search",
  "action_input": "iphone 15"
}
```
Observation: Found: iphone 15 (80,000 INR), Samsung S24 (75000 INR), all in stock.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The iPhone 15 costs 80,000 INR and is in stock."
}
```

> Finished chain.


'The iPhone 15 costs 80,000 INR and is in stock.'

In [9]:
manager.run("Whats the return on accessories?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "policy_search",
  "action_input": "return policy for accessories"
}
```
Observation: Policy : 7-day return for phones, no returns for accessories.
Thought:I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "There are no returns for accessories."
}
```

> Finished chain.


'There are no returns for accessories.'

In [10]:
manager.run("Whats the price of iphone 15 and return policy??")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "product_search",
  "action_input": "iphone 15 price"
}
```
Observation: Found: iphone 15 (80,000 INR), Samsung S24 (75000 INR), all in stock.
Thought:Action:
```
{
  "action": "policy_search",
  "action_input": "iphone 15 return policy"
}
```
Observation: Policy : 7-day return for phones, no returns for accessories.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The iPhone 15 costs 80,000 INR and is in stock. The return policy for phones is 7 days."
}
```

> Finished chain.


'The iPhone 15 costs 80,000 INR and is in stock. The return policy for phones is 7 days.'

In [11]:
# ==========================================
# CELL 3: DECENTRALIZED ARCHITECTURE (The Hand-off)
# ==========================================
from langchain.agents import initialize_agent, AgentType, Tool

# --- Step 1: Create Individual Agents ---
# We use ZERO_SHOT_REACT_DESCRIPTION because it handles simple string inputs better.

product_agent = initialize_agent(
    tools=[product_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # <--- CHANGED THIS
    verbose=True,
    handle_parsing_errors=True # Helps agent recover if it messes up formatting
)

policy_agent = initialize_agent(
    tools=[policy_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # <--- CHANGED THIS
    verbose=True,
    handle_parsing_errors=True
)

# --- Step 2: The Hand-off Logic (The "Glue") ---
def run_decentralized_flow(query):
    print(f"User Query: {query}")
    print("👉 Sending to Product Agent first...")

    try:
        # 1. Attempt with Product Agent
        response = product_agent.run(query)

        # 2. SIMULATED CHECK: The agent might answer "I don't know", but for
        # this demo, we force a hard failure if it's the wrong domain.
        if "return" in query.lower() or "policy" in query.lower():
            raise ValueError("Not my expertise!")

        return response

    except Exception as e:
        # 3. The Fallback (Hand-off)
        print(f"\n❌ Product Agent refused: {e}")
        print("🔄 Handing off to Policy Agent...")
        return policy_agent.run(query)

# --- Step 3: Run It ---
print("\n🤖 DECENTRALIZED SYSTEM: Passing the baton.")
final_answer = run_decentralized_flow("What is the return policy?")
print(f"\n📢 Final Answer: {final_answer}")


🤖 DECENTRALIZED SYSTEM: Passing the baton.
User Query: What is the return policy?
👉 Sending to Product Agent first...


> Entering new AgentExecutor chain...
I cannot answer questions about return policies with the available tools. The `product_search` tool is designed for querying product price and stock information, not general store policies.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I cannot answer questions about return policies with the available tools. The `product_search` tool is designed for querying product price and stock information, not general store policies.
Final Answer: I cannot answer questions about return policies with the available tools.

> Finished chain.

❌ Product Agent refused: Not my expertise!
🔄 Handing off to Policy Agent...


> Entering new AgentExecutor chain...
Action: policy_search
Action Input:
Observation: Policy : 7-day return for phones, no returns for accessories.
Thought:I now know the final answer
Final Answer: The r

In [12]:
# Hybrid Architecture (The Smart Router)
# LLM decide - who is the best to answer this question?

# Step 1 :
def smart_router_and_execute(query):
  router_prompt = f"""
  Classify this query into exactly 1 category : PRODUCT, POLICY or GENERAL.
  Query : {query}
  Rules:
  - PRODUCT: Buying, prices, stock, device specs.
  - POLICY: Returns, refunds, warranty, shipping, support.
  - GENERAL: Greetings, how are you, general help.

  Return ONLY the category word.
"""
  category = llm.invoke(router_prompt).content.strip().upper()


  # Step 2 : Dispatching for execution
  # we re-use the agents we created in the step above!
  if "PRODUCT" in category :
    return product_agent.run(query)

  elif "POLICY" in category :
    return policy_agent.run(query)

  else :
    return "Hi, please ask questions only about policies / prices."

In [13]:
smart_router_and_execute("I want to buy a Samsung phone, how much is it?")



> Entering new AgentExecutor chain...
I need more information to answer this question. Samsung makes many different models of phones, and their prices vary widely. Could you please specify which Samsung phone model you are interested in?
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I need more information to answer this question. Samsung makes many different models of phones, and their prices vary widely. Could you please specify which Samsung phone model you are interested in?
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Thought: The user wants to know the price of a Samsung phone. Samsung produces many different models of phones, and their prices vary significantly. To provide an accurate answer, I need a more specific model. However, the format requires an action. I will attempt a broad `product_search` for "Samsung phone" to see if the tool can handle such a general query or if it provides a range of products.
Action: product_se

'A Samsung S24 costs 75000 INR and is in stock.'

In [14]:
def smart_router_and_execute(query):
  router_prompt = f"""
  Classify this query into exactly 1 category : PRODUCT, POLICY or GENERAL.
  Query : {query}
  Rules:
  - PRODUCT: Buying, prices, stock, device specs.
  - POLICY: Returns, refunds, warranty, shipping, support.
  - GENERAL: Greetings, how are you, general help.

  Return ONLY the category word.
"""
  category = llm.invoke(router_prompt).content.strip().upper()
  print(category)

In [15]:
smart_router_and_execute("My iphone is broken, i need to return it and I want to buy a Samsung phone, how much is it?")

POLICY
